# Big Data Platforms - Home Assignment 1
## Omri Newman SID806646
Due 13.11.2021


In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import random
import os
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.csv as csv
import dask.dataframe as dd

## Task 1
Create local CSV file “mydata.csv”  with 1000000 rows with columns (id, fruit, price, color). Use random value for rows, where fruit has one of the values ['Orange', 'Grape', 'Apple', 'Banana', 'Pineapple', 'Avocado'] and colors are   ['Red', 'Green', 'Yellow', 'Blue']. Price should be random integer between 10 and 100. Filed id should be an index number starting from 1.

In [2]:
def check_csv():
    """check if 'mydata.csv' exists in current working directory.
    If not, create 'mydata.csv'.
    """
    path = os.path.join(os.getcwd(), "mydata.csv")
    if os.path.isfile(path):
        print("mydata.csv already exists")
        pass
    else:
        index = range(1, 1000001)
        cost = range(10, 101)
        fruit = [random.choice(
            ["Orange", "Grape", "Apple", "Banana", "Pineapple", "Avocado"]) for
                 i in index]
        price = [random.choice(cost) for i in index]
        color = [random.choice(["Red", "Green", "Yellow", "Blue"]) for i in
                 index]
        df = pd.DataFrame(
            {"id": index, "fruit": fruit, "price": price, "color": color})
        df.to_csv("mydata.csv", index=False)
        print("creating mydata.csv")

if __name__ == '__main__':
    check_csv()    

mydata.csv already exists


Write Python code to create SQLite database “mydb.db” and create a table “mydata” with the schema of the “mydata.csv” 

In [3]:
def check_mydatabase():
    """check if 'mydb.db' database exists in current working directory. 
    If not, create it."""
    path = os.path.join(os.getcwd(), "mydb.db")
    if os.path.isfile(path):
        print("sqlite version: " + str(sqlite3.version))
        print("mydb.db already exists")
        pass
    else:
        conn = sqlite3.connect('mydb.db')
        print("creating mydb.db")
        
def check_mydata():
    """check if 'mydata' table exists in 'mydb.db' database. 
    If not, create it."""
    path = os.path.join(os.getcwd(), "mydb.db")
    filesize = os.path.getsize(path)
    if filesize != 0:
        print("mydata already exists")
        pass
    else:
        conn = sqlite3.connect('mydb.db')
        c = conn.cursor()
        c.execute(''' CREATE TABLE mydata
        (id INT,
         fruit VARCHAR(20),
         price INT,
         color VARCHAR(20))
        ''')
        conn.commit()
        print("creating mydata")

if __name__ == '__main__':
    check_mydatabase()

if __name__ == '__main__':
    check_mydata()

sqlite version: 2.6.0
mydb.db already exists
mydata already exists


Write Python code to load “mydata.csv” into “mydata” table.

In [4]:
def load_csv():
    """Check if 'mydata.csv' has been loaded into 'mydata' in 'mydb.db'. If not, append into the schema."""
    path = os.path.join(os.getcwd(), "mydb.db")
    filesize = os.path.getsize(path)
    if filesize < 14000000:
        conn = sqlite3.connect('mydb.db')
        df = pd.read_csv("mydata.csv")
        df.to_sql('mydata', conn, if_exists='append', index=False)
        print("loading mydata.csv to mydata in mydb.db")
    else:
        print("mydata.csv already loaded")
        pass

if __name__ == '__main__':
    load_csv()

mydata.csv already loaded


Write 2 different SQL statements  with different conditions to retrieve different rows. Explain which parts of the statement are predicate and which parts are projection.

In [5]:
"""Predicate refers to the where/filter clause which effects the amount of rows returned, and selects rows 
that specify a given logic. 

Projection refers to the selected columns when wanting to see a few attributes instead 
of all attributes.

This query Selects all columns from mydata which is a Projection. It then orders all the rows according to 
their price, which is a Predicate."""

conn = sqlite3.connect('mydb.db')

query = ('''SELECT * 
          FROM mydata
          ORDER BY price ASC''')

pd.read_sql_query(query, conn)

,id,fruit,price,color
0,64,Pineapple,10,Yellow
1,72,Avocado,10,Green
2,126,Pineapple,10,Yellow
3,149,Pineapple,10,Yellow
4,268,Pineapple,10,Yellow
...,...,...,...,...
999995,999676,Orange,100,Red
999996,999748,Avocado,100,Yellow
999997,999838,Apple,100,Yellow
999998,999869,Banana,100,Red


In [6]:
"""Predicate refers to the where/filter clause which effects the amount of rows returned, and selects rows 
that specify a given logic. 

Projection refers to the selected columns when wanting to see a few attributes instead 
of all attributes.

This query Selects all columns from mydata which is a Projection. But it selects a subset of rows, namely 
it returns fruit that start with the letter 'B' and colors that start with the letter 'Y', it then orders
according to price - Predicate."""

conn = sqlite3.connect('mydb.db')

query = ('''SELECT id, fruit, price, color 
          FROM mydata
          WHERE fruit LIKE 'B%' AND color LIKE 'Y%'
          ORDER BY price ASC''')

pd.read_sql_query(query, conn, index_col='id')

,fruit,price,color
id,,,
864,Banana,10,Yellow
1632,Banana,10,Yellow
1926,Banana,10,Yellow
6780,Banana,10,Yellow
7798,Banana,10,Yellow
...,...,...,...
981982,Banana,100,Yellow
989804,Banana,100,Yellow
992060,Banana,100,Yellow


## Task 2:  CSV and Parquet
Write Python program that reads “mydata.csv” file and count number of lines

In [7]:
df = pd.read_csv("mydata.csv")
print("mydata.csv has %d lines" % (len(df)+1))

mydata.csv has 1000001 lines


By using PyArrow, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapyarrow.parquet”

In [8]:
df = csv.read_csv('mydata.csv')
pq.write_table(df, 'mydatapyarrow.parquet')

By using Dask, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatadask.parquet”

In [9]:
df = dd.read_csv('mydata.csv').set_index('id')
df.to_parquet('mydatadask.parquet')

(None,)

By using Pandas, create Parquet file from the “mydata.csv”. Name Parquet file as “mydatapandas.parquet”

In [10]:
df = pd.read_csv("mydata.csv")
df.to_parquet('mydatapandas.parquet')

Examine generated Parquet files.  Why do you think Dask generated Parquet file differently than PyArrow and Pandas? What might be explanation for this?

*Dask is written for parallel computing while PyArrow and Pandas are not which I imagine is why it generated 
the Parquet file differently. Dask turned the file into its own directory containing metadata from the file.* 


## Task 3: Split CSV files

Write Python code that calculates size of “mydata.csv” in bytes. Define an integer variable “middle” which is the size of “mydata.csv”  divided by 2. 

In [12]:
path = os.path.join(os.getcwd(), "mydata.csv")
filesize = os.path.getsize(path)
middle = filesize//2
print("mydata.csv has %d bytes" % (filesize))
print("The middle 'byte' is %d" % (middle))

mydata.csv has 22733982 bytes
The middle 'byte' is 11366991


Write a Python function first_chunk that count number of rows by reading the byte range of the CSV file, from 0 till the “middle”. Write a function last_chunk that count number of rows by reading byte range of CSV file from the “middle”+1 till the end of the file. Notice: Use seek to position into middle + 1 location. To correctly seek and read, you need to open file into binary mode and use decoder, for example to read the second chunk

In [13]:
def first_chunk(path):
    filesize = os.path.getsize(path)
    middle = int(filesize//2)
    file = open(path, "rb")
    r = file.read(middle+1).decode(encoding="utf-8")
    lst = r.split("\n")     
    return len(lst)

def second_chunk(path):
    filesize = os.path.getsize(path)
    middle = int(filesize//2)
    file = open(path, "rb")
    file.seek(middle+1)
    r = file.read().decode(encoding="utf-8")
    lst = r.split("\n")
    if len(lst[-1]) == 0:
        lst.pop()
    return len(lst)
    

In [14]:
first_chunk("mydata.csv")+second_chunk("mydata.csv")

1000002

Explain why total number of lines from the first chunk and second chunk is larger than the number of lines calculated in the step (1) of Task 2.

*The number of lines from the first chunk and second chunk combined is larger than what was calculated in 
Task 2 because in most cases, the split line will be counted twice by both functions.*

Suggest an algorithm to resolve the issue from the step (3) and implement it.

*You could simply use the same algorithm but add an if statement in first_chunk() which checks if the middle byte 
is a new line character, and if so returns the len of the list. If not it returns the length of the list minus 
one since that will always reconcile the double count from the middle line.* 

In [19]:
def new_first_chunk(path):
    filesize = os.path.getsize(path)
    middle = int(filesize//2)
    file = open(path, "rb")
    r = file.read(middle+1).decode(encoding="utf-8")
    lst = r.split("\n")     
    if len(lst[-1]) == 0:
        lst.pop()
        return len(lst)
    return len(lst)-1

In [20]:
new_first_chunk("mydata.csv")+second_chunk("mydata.csv")

1000001

Check the algorithm of step (4) with multiple chunks. Define a chunk size to be 16MB. Write a function that process “mydata.csv “ in chunks and count number of lines for each chunk. For example, first chunk will be 0-16MB, second chunk 16MB-32BM, and so on, until the last chunk, which might be smaller.

In [21]:
def complete_chunks(path, size):
    filesize = os.path.getsize(path)
    complete_chunk = filesize//size
    complete_lines = []
    for i in range(complete_chunk): 
        file = open(path, "rb")  
        file.seek(((i)*size))
        r = file.read(size).decode(encoding="utf-8")
        lst = r.split("\n")
        if len(lst[-1]) == 0:
            lst.pop()
            complete_lines.append(len(lst))
        else:
            complete_lines.append(len(lst)-1) 
    return sum(complete_lines), complete_chunk

def sub_chunk(path, size):
    filesize = os.path.getsize(path)
    complete_chunk = filesize//size
    sub_chunk = filesize%size
    file = open(path, "rb")
    file.seek(int(complete_chunk*size)+1)
    r = file.read().decode(encoding="utf-8")
    lst = r.split("\n")
    if len(lst[-1]) == 0:
        lst.pop()
    return len(lst), sub_chunk

In [22]:
complete_chunks("mydata.csv", 16000000)
print("There is %d complete 16mb chunk in mydata.csv, which consist of %d lines out of 1000001" % 
      (complete_chunks("mydata.csv", 16000000)[1], complete_chunks("mydata.csv", 16000000)[0]))

There is 1 complete 16mb chunk in mydata.csv, which consist of 705160 lines out of 1000001


In [23]:
print("The remaining chunk consists of %d bytes and accounts for %d lines out of 1000001" %
     (sub_chunk("mydata.csv", 16000000)[1], sub_chunk("mydata.csv", 16000000)[0]))

The remaining chunk consists of 6733982 bytes and accounts for 294841 lines out of 1000001


In [24]:
# The bytes add up to the filesize 
(16000000+sub_chunk("mydata.csv", 16000000)[1]) == os.path.getsize("mydata.csv")

True

In [25]:
complete_chunks("mydata.csv", 16000000)[0] + sub_chunk("mydata.csv", 16000000)[0]

1000001

### Test the above function on other chunk sizes..

In [33]:
complete_chunks("mydata.csv", 5000000)
print("There are %d complete 5mb chunks in mydata.csv which consist of %d out of 1000001 lines" % 
      (complete_chunks("mydata.csv", 5000000)[1], complete_chunks("mydata.csv", 5000000)[0]))

There are 4 complete 5mb chunks in mydata.csv which consist of 880330 out of 1000001 lines


In [30]:
print("The remaining chunk consists of %d bytes and accounts for %d lines out of 1000001" %
     (sub_chunk("mydata.csv", 5000000)[1], sub_chunk("mydata.csv", 5000000)[0]))

The remaining chunk consists of 2733982 bytes and accounts for 119671 lines out of 1000001


In [31]:
complete_chunks("mydata.csv", 5000000)[0] + sub_chunk("mydata.csv", 5000000)[0]

1000001